In [1]:
import pandas as pd
from datetime import datetime, timedelta
from collections import OrderedDict
import numpy as np

In [2]:
df_charge = pd.read_excel('/Users/j-yeom/workspace/asakai/charge/charge-total.xlsx', sheet_name = ['03-03','2018-04','2018-05','2018-06'])
#print(df)
df1_charge = pd.DataFrame()
df1_charge['date'] = (df_charge['03-03']['date']).append(df_charge['2018-04']['date'])
df1_charge['date'] = pd.to_datetime(df1_charge['date'], format='%Y/%m/%d %H:%M')
df1_charge['date_month'] = df1_charge['date'].dt.strftime('%Y-%m')
df1_charge['userId'] = (df_charge['03-03']['userId']).append(df_charge['2018-04']['userId'])
df1_charge['userId'] = df1_charge.userId.apply(str)

In [3]:
# import data
df_trans = pd.read_csv('/Users/j-yeom/workspace/asakai/pollet-renamed2.csv')
# add monthly column
df_trans['date'] = pd.to_datetime(df_trans['date'], format='%Y/%m/%d %H:%M')
df_trans['date_month'] = df_trans['date'].dt.strftime('%Y-%m')
df_trans['userId'] = df_trans.userId.apply(str)
df_trans['march'] = df_trans['userId'].where(df_trans['date_month'] == '2017-03')
monthList = df_trans.date_month.unique()
userList = []

In [4]:
monthList = df1_charge.date_month.unique()
#print(monthList)
userList = []
for month in monthList:
    if(len(userList) == 0):
        userList.append(list(df1_charge.userId.where(df1_charge['date_month'] == month).unique()))
    else:
        toAppend = set(df1_charge.userId.where(df1_charge['date_month'] == month).unique())
        # remove all the previous months' users
        toRemove = set().union(*userList)
        userList.append(list(toAppend - toRemove))
#print(userList) # userList will now contain lists of first users from each month.
userObj = {}
count = 0
for month in monthList:
    userObj[month] = userList[count]
    count+=1
# print(userObj)

In [5]:
userList = {}
for month in monthList:
    charge_user = df1_charge.userId.where(df1_charge['date_month'] == month).unique()  # << unique user of each month
    trans_user = df_trans.userId.where(df_trans['date_month'] == month).unique()
    userList[month] = list(set(charge_user).union(set(trans_user)))
#print(userList) # 각 월의 유저아이디를 담고 있는 객체

def match_all(rangeArr, subRangeArr):
    arr = {}
    for month in rangeArr:
        arr[month] = []
        for sub_month in subRangeArr:  
            # figure out if each month unique user existed in the first charger user group
            arr[month].append(np.isin(userList[sub_month],userObj[month]).sum())
    return arr

dates = ["2017-03-01", "2018-04-30"]
start, end = [datetime.strptime(_, "%Y-%m-%d") for _ in dates]
result = list(OrderedDict(((start + timedelta(_)).strftime(r"%Y-%m"), None) for _ in range((end - start).days)).keys())
#print(for_march(result))
real_result = match_all(result, result) # use same time range for both compare-ref and compare-ee
print(real_result)

{'2017-03': [769, 251, 236, 253, 222, 250, 255, 185, 213, 271, 115, 109, 119, 126], '2017-04': [0, 507, 159, 167, 124, 142, 145, 116, 114, 165, 73, 71, 77, 81], '2017-05': [0, 0, 771, 246, 175, 196, 197, 160, 188, 259, 121, 118, 136, 132], '2017-06': [0, 0, 0, 415, 119, 134, 105, 75, 74, 132, 49, 48, 59, 65], '2017-07': [0, 0, 0, 0, 390, 163, 146, 110, 111, 126, 83, 93, 93, 81], '2017-08': [0, 0, 0, 0, 0, 790, 203, 125, 125, 153, 75, 83, 92, 93], '2017-09': [0, 0, 0, 0, 0, 0, 509, 111, 122, 128, 52, 57, 57, 61], '2017-10': [0, 0, 0, 0, 0, 0, 0, 289, 125, 111, 50, 54, 58, 44], '2017-11': [0, 0, 0, 0, 0, 0, 0, 0, 242, 122, 43, 53, 50, 47], '2017-12': [0, 0, 0, 0, 0, 0, 0, 0, 0, 691, 194, 165, 170, 133], '2018-01': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 719, 206, 210, 158], '2018-02': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 297, 104, 81], '2018-03': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 397, 142], '2018-04': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 411]}


In [9]:
df2 = df1_charge.groupby('date_month')[['userId']].nunique()
for month, month_list in real_result.items():
    df2[month] = month_list
#df2 = df2.transpose()
df2 = df2.replace(0, np.nan)
#    return
print(df2)
writer = pd.ExcelWriter('pollet_charge_trans1.xlsx', engine='xlsxwriter')
df2.to_excel(writer, sheet_name='Sheet1')
writer.save()
#df2.to_json('pollet_charge_trans.json', orient='split')

            userId  2017-03  2017-04  2017-05  2017-06  2017-07  2017-08  \
date_month                                                                 
2017-03        768      769      NaN      NaN      NaN      NaN      NaN   
2017-04        757      251    507.0      NaN      NaN      NaN      NaN   
2017-05       1165      236    159.0    771.0      NaN      NaN      NaN   
2017-06       1080      253    167.0    246.0    415.0      NaN      NaN   
2017-07       1029      222    124.0    175.0    119.0    390.0      NaN   
2017-08       1674      250    142.0    196.0    134.0    163.0    790.0   
2017-09       1559      255    145.0    197.0    105.0    146.0    203.0   
2017-10       1170      185    116.0    160.0     75.0    110.0    125.0   
2017-11       1313      213    114.0    188.0     74.0    111.0    125.0   
2017-12       2157      271    165.0    259.0    132.0    126.0    153.0   
2018-01       1573      115     73.0    121.0     49.0     83.0     75.0   
2018-02     